#### Загрузка и изучение данных

In [1]:
pip install scikit-learn==1.3.2 -q 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install shap -q 

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xgboost lightgbm catboost -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install seaborn -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import re
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from scipy.stats import shapiro
from pandas.api.types import is_integer_dtype
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score
import shap
from typing import Optional, List

import warnings
warnings.filterwarnings('ignore')

In [6]:
RANDOM_STATE = 42
TEST_SIZE = 0.25
TARGET_COL = "heart_attack_risk_binary"

In [7]:
heart_train = pd.read_csv('C:/Users/malen/OneDrive/Desktop/data_since/мастерская 1/data/raw/heart_train.csv', index_col='id')
heart_train.head()

Unnamed: 0       Age  Cholesterol  Heart rate  Diabetes  Family History  \
id                                                                              
2664           0  0.359551     0.732143    0.074244       1.0             1.0   
9287           1  0.202247     0.325000    0.047663       1.0             1.0   
5379           2  0.606742     0.860714    0.055912       1.0             0.0   
8222           3  0.730337     0.007143    0.053162       0.0             0.0   
4047           4  0.775281     0.757143    0.021998       0.0             0.0   

      Smoking  Obesity  Alcohol Consumption  Exercise Hours Per Week  ...  \
id                                                                    ...   
2664      1.0      1.0                  1.0                 0.535505  ...   
9287      0.0      0.0                  1.0                 0.068690  ...   
5379      1.0      1.0                  1.0                 0.944001  ...   
8222      1.0      0.0                  1.0                 0.697023  ...   
4047      1.0      0.0                  1.0                 0.412878  ...   

      Triglycerides  Physical Activity Days Per Week  Sleep Hours Per Day  \
id                                                                          
2664       0.979221                              3.0             0.333333   
9287       0.515584                              3.0             0.833333   
5379       0.012987                              2.0             1.000000   
8222       0.131169                              0.0             0.333333   
4047       0.070130                              5.0             1.000000   

      Heart Attack Risk (Binary)  Blood sugar     CK-MB  Troponin  Gender  \
id                                                                          
2664                         0.0     0.227018  0.048229  0.036512    Male   
9287                         0.0     0.150198  0.017616  0.000194  Female   
5379                         0.0     0.227018  0.048229  0.036512  Female   
8222                         1.0     0.227018  0.048229  0.036512  Female   
4047                         1.0     0.227018  0.048229  0.036512    Male   

      Systolic blood pressure  Diastolic blood pressure  
id                                                       
2664                 0.212903                  0.709302  
9287                 0.412903                  0.569767  
5379                 0.238710                  0.220930  
8222                 0.348387                  0.267442  
4047                 0.619355                  0.441860  

[5 rows x 27 columns]

In [8]:
heart_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8685 entries, 2664 to 7270
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       8685 non-null   int64  
 1   Age                              8685 non-null   float64
 2   Cholesterol                      8685 non-null   float64
 3   Heart rate                       8685 non-null   float64
 4   Diabetes                         8442 non-null   float64
 5   Family History                   8442 non-null   float64
 6   Smoking                          8442 non-null   float64
 7   Obesity                          8442 non-null   float64
 8   Alcohol Consumption              8442 non-null   float64
 9   Exercise Hours Per Week          8685 non-null   float64
 10  Diet                             8685 non-null   int64  
 11  Previous Heart Problems          8442 non-null   float64
 12  Medication Use        

heart_train состоит из 8685 записей и 27 столбцов, 243 пропуска в отдельных столбцах. Требуется изменение типа данных object.

In [9]:
heart_test = pd.read_csv('C:/Users/malen/OneDrive/Desktop/data_since/мастерская 1/data/raw/heart_test.csv', index_col='id')
heart_test.head()

Unnamed: 0       Age  Cholesterol  Heart rate  Diabetes  Family History  \
id                                                                              
7746           0  0.494382     0.264286    0.062328       0.0             1.0   
4202           1  0.224719     0.953571    0.082493       1.0             0.0   
6632           2  0.629213     0.092857    0.064161       0.0             1.0   
4639           3  0.460674     0.567857    0.055912       1.0             1.0   
4825           4  0.719101     0.485714    0.022915       1.0             0.0   

      Smoking  Obesity  Alcohol Consumption  Exercise Hours Per Week  ...  \
id                                                                    ...   
7746      1.0      1.0                  1.0                 0.361618  ...   
4202      0.0      1.0                  0.0                 0.996483  ...   
6632      1.0      1.0                  0.0                 0.995561  ...   
4639      1.0      1.0                  1.0                 0.437277  ...   
4825      1.0      0.0                  1.0                 0.514920  ...   

           BMI  Triglycerides  Physical Activity Days Per Week  \
id                                                               
7746  0.283490       0.306494                              1.0   
4202  0.467036       0.087013                              0.0   
6632  0.409366       0.205195                              7.0   
4639  0.910261       0.163636                              0.0   
4825  0.758924       0.580519                              5.0   

      Sleep Hours Per Day  Blood sugar     CK-MB  Troponin  Gender  \
id                                                                   
7746             0.333333     0.227018  0.048229  0.036512    Male   
4202             0.166667     0.227018  0.048229  0.036512  Female   
6632             1.000000     0.102767  0.002666  0.088455    Male   
4639             0.666667     0.203557  0.056390  0.271774  Female   
4825             0.000000     0.227018  0.048229  0.036512    Male   

      Systolic blood pressure  Diastolic blood pressure  
id                                                       
7746                 0.283871                  0.372093  
4202                 0.703226                  0.441860  
6632                 0.458065                  0.779070  
4639                 0.741935                  0.255814  
4825                 0.412903                  0.395349  

[5 rows x 26 columns]

In [10]:
heart_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966 entries, 7746 to 6521
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       966 non-null    int64  
 1   Age                              966 non-null    float64
 2   Cholesterol                      966 non-null    float64
 3   Heart rate                       966 non-null    float64
 4   Diabetes                         935 non-null    float64
 5   Family History                   935 non-null    float64
 6   Smoking                          935 non-null    float64
 7   Obesity                          935 non-null    float64
 8   Alcohol Consumption              935 non-null    float64
 9   Exercise Hours Per Week          966 non-null    float64
 10  Diet                             966 non-null    int64  
 11  Previous Heart Problems          935 non-null    float64
 12  Medication Use         

heart_test состоит из 8685 записей и 26 столбцов, 31 пропуск в отдельных столбцах. Требуется изменение типа данных object.

In [11]:
heart_train['Unnamed: 0'].value_counts()

Unnamed: 0
0       1
5785    1
5799    1
5798    1
5797    1
       ..
2890    1
2889    1
2888    1
2887    1
8684    1
Name: count, Length: 8685, dtype: int64

In [12]:
heart_test['Unnamed: 0'].value_counts()

Unnamed: 0
0      1
604    1
636    1
637    1
638    1
      ..
326    1
327    1
328    1
329    1
965    1
Name: count, Length: 966, dtype: int64

данный столбец использовался для нумерации строк в таблице, поэтому он нам не нужен.

In [13]:
heart_train = heart_train.drop('Unnamed: 0', axis=1)
heart_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8685 entries, 2664 to 7270
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              8685 non-null   float64
 1   Cholesterol                      8685 non-null   float64
 2   Heart rate                       8685 non-null   float64
 3   Diabetes                         8442 non-null   float64
 4   Family History                   8442 non-null   float64
 5   Smoking                          8442 non-null   float64
 6   Obesity                          8442 non-null   float64
 7   Alcohol Consumption              8442 non-null   float64
 8   Exercise Hours Per Week          8685 non-null   float64
 9   Diet                             8685 non-null   int64  
 10  Previous Heart Problems          8442 non-null   float64
 11  Medication Use                   8442 non-null   float64
 12  Stress Level          

In [14]:
heart_test = heart_test.drop('Unnamed: 0', axis=1)
heart_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966 entries, 7746 to 6521
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              966 non-null    float64
 1   Cholesterol                      966 non-null    float64
 2   Heart rate                       966 non-null    float64
 3   Diabetes                         935 non-null    float64
 4   Family History                   935 non-null    float64
 5   Smoking                          935 non-null    float64
 6   Obesity                          935 non-null    float64
 7   Alcohol Consumption              935 non-null    float64
 8   Exercise Hours Per Week          966 non-null    float64
 9   Diet                             966 non-null    int64  
 10  Previous Heart Problems          935 non-null    float64
 11  Medication Use                   935 non-null    float64
 12  Stress Level           

#### Предобработка данных

Выполним предобработку данных. Изменим названия столбцов, типы данных, где это необходимо. Проведем проверку на явные и неявные дубликаты в данных.

##### Изменение наименования столбцов

Приведем наименования столбцов к змеиному регистру

In [15]:
def to_snake_case(name):
    name = name.strip()
    name = re.sub(r'[\s\-]+', '_', name)               # пробелы и дефисы → _
    name = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', name) # camelCase → camel_case
    name = re.sub(r'[^\w_]', '', name)                  # убрать лишние символы
    return name.lower()

heart_train.columns = [to_snake_case(c) for c in heart_train.columns]
heart_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8685 entries, 2664 to 7270
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   age                              8685 non-null   float64
 1   cholesterol                      8685 non-null   float64
 2   heart_rate                       8685 non-null   float64
 3   diabetes                         8442 non-null   float64
 4   family_history                   8442 non-null   float64
 5   smoking                          8442 non-null   float64
 6   obesity                          8442 non-null   float64
 7   alcohol_consumption              8442 non-null   float64
 8   exercise_hours_per_week          8685 non-null   float64
 9   diet                             8685 non-null   int64  
 10  previous_heart_problems          8442 non-null   float64
 11  medication_use                   8442 non-null   float64
 12  stress_level          

In [16]:
heart_test.columns = [to_snake_case(c) for c in heart_test.columns]
heart_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966 entries, 7746 to 6521
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   age                              966 non-null    float64
 1   cholesterol                      966 non-null    float64
 2   heart_rate                       966 non-null    float64
 3   diabetes                         935 non-null    float64
 4   family_history                   935 non-null    float64
 5   smoking                          935 non-null    float64
 6   obesity                          935 non-null    float64
 7   alcohol_consumption              935 non-null    float64
 8   exercise_hours_per_week          966 non-null    float64
 9   diet                             966 non-null    int64  
 10  previous_heart_problems          935 non-null    float64
 11  medication_use                   935 non-null    float64
 12  stress_level           

##### Обработка пропусков

In [17]:
heart_train[heart_train['stress_level'].isna()]

age  cholesterol  heart_rate  diabetes  family_history  smoking  \
id                                                                           
9528  0.449438      0.49978    0.063245       NaN             NaN      NaN   
9640  0.370787      0.49978    0.066911       NaN             NaN      NaN   
9515  0.629213      0.49978    0.073327       NaN             NaN      NaN   
9491  0.516854      0.49978    0.045830       NaN             NaN      NaN   
9448  0.269663      0.49978    0.035747       NaN             NaN      NaN   
...        ...          ...         ...       ...             ...      ...   
9455  0.719101      0.49978    0.049496       NaN             NaN      NaN   
9506  0.651685      0.49978    0.036664       NaN             NaN      NaN   
9474  0.651685      0.49978    0.054079       NaN             NaN      NaN   
9467  0.606742      0.49978    0.059578       NaN             NaN      NaN   
9555  0.629213      0.49978    0.057745       NaN             NaN      NaN   

      obesity  alcohol_consumption  exercise_hours_per_week  diet  ...  \
id                                                                 ...   
9528      NaN                  NaN                  0.50211     3  ...   
9640      NaN                  NaN                  0.50211     3  ...   
9515      NaN                  NaN                  0.50211     3  ...   
9491      NaN                  NaN                  0.50211     3  ...   
9448      NaN                  NaN                  0.50211     3  ...   
...       ...                  ...                      ...   ...  ...   
9455      NaN                  NaN                  0.50211     3  ...   
9506      NaN                  NaN                  0.50211     3  ...   
9474      NaN                  NaN                  0.50211     3  ...   
9467      NaN                  NaN                  0.50211     3  ...   
9555      NaN                  NaN                  0.50211     3  ...   

      triglycerides  physical_activity_days_per_week  sleep_hours_per_day  \
id                                                                          
9528       0.503603                              NaN             0.504621   
9640       0.503603                              NaN             0.504621   
9515       0.503603                              NaN             0.504621   
9491       0.503603                              NaN             0.504621   
9448       0.503603                              NaN             0.504621   
...             ...                              ...                  ...   
9455       0.503603                              NaN             0.504621   
9506       0.503603                              NaN             0.504621   
9474       0.503603                              NaN             0.504621   
9467       0.503603                              NaN             0.504621   
9555       0.503603                              NaN             0.504621   

      heart_attack_risk_binary  blood_sugar     ck_mb  troponin  gender  \
id                                                                        
9528                       0.0     0.233202  0.026692  0.000777     1.0   
9640                       0.0     0.154150  0.004702  0.042043     1.0   
9515                       0.0     0.256917  0.012310  0.000777     0.0   
9491                       0.0     0.559289  0.006704  0.000291     0.0   
9448                       0.0     0.112648  0.051218  0.000485     0.0   
...                        ...          ...       ...       ...     ...   
9455                       0.0     0.138340  0.004034  0.003204     1.0   
9506                       0.0     0.239130  0.055790  0.060783     1.0   
9474                       0.0     0.276680  0.008673  0.150403     1.0   
9467                       0.0     0.156126  0.057692  0.000583     0.0   
9555                       0.0     0.148221  0.004001  0.067968     1.0   

      systolic_blood_pressure  diastolic_blood_pressure

В обучающем наборе данных пропуски встречаются только в 9 конкретных признаках:
diabetes, family_history, smoking, obesity, alcohol_consumption,
physical_activity_days_per_week, previous_heart_problems, medication_use, stress_level.

Все они связаны с образом жизни и историей болезни пациента.
Количество пропусков одинаковое во всех этих колонках — 243 записи (≈ 2,8% датасета). Это значит, что пропуски не случайны, а возникают «пакетом» для целых строк, что, вероятно, связано с особенностями сбора данных (например, неполные анкеты или пациенты, которым не задавали эти вопросы).

Потеря 2,8% строк снизит размер обучающей выборки и может убрать целую подгруппу пациентов, что приведёт к смещению распределения признаков.

В тестовых данных возможны такие же паттерны пропусков, и если мы их выбросим из train, модель не научится корректно обрабатывать такие случаи.

Пропуски содержат потенциально полезную информацию — сам факт, что данные не заполнены, может коррелировать с вероятностью сердечного приступа (например, пациенты с неполной медицинской историей).

Стратегия обработки NaN:
- Сохранить все строки, включая те, где есть пропуски.
- Добавить индикаторные признаки __was_missing для каждого из 9 признаков с пропусками — 1, если значение отсутствует, и 0, если присутствует. Это позволит модели учитывать сам факт пропуска как сигнал.
- Импутировать пропуски. Для бинарных признаков (0/1) — мода по обучающим данным. Для порядковых признаков (stress_level, physical_activity_days_per_week) — медиана. Для остальных числовых признаков — медиана.

Такой подход минимизирует потерю информации, сохраняет распределения признаков, даёт модели возможность использовать признак «пропуск» как фактор прогноза, и при этом обеспечивает готовность данных для алгоритмов, которые не работают с NaN напрямую.

##### Обработка дубликатов

Перед построением модели прогнозирования риска сердечного приступа необходимо провести предварительную проверку качества исходных данных. Цель — выявить и устранить записи, которые могут исказить обучение модели: полные дубликаты строк, дубликаты с противоречивыми значениями целевого признака, а также неявные повторы. Это позволяет снизить риск переобучения, исключить влияние повторяющихся пациентов и повысить обобщающую способность модели.

Проверка на дубликаты выполняется в несколько этапов:
1. Полные дубликаты
- Ищем строки, полностью совпадающие по всем признакам (включая целевой признак).
- Если такие строки есть, оставляем только первую, остальные удаляем.
- Это исключает ситуации, когда один и тот же пациент учитывается несколько раз без изменений данных.

2. Дубликаты по признакам с разными значениями целевого признака (конфликтные записи)
- Определяем группы записей, полностью совпадающих по всем признакам, кроме целевого (heart_attack_risk_binary).
- Если внутри группы встречаются разные значения целевого признака, удаляем все такие записи.
- Это предотвращает попадание в обучающую выборку противоречивой информации, которая мешает алгоритму обучаться.

3. Дубликаты по признакам с одинаковым целевым признаком (неявные дубликаты)
- Если записи совпадают по всем признакам и целевому признаку, но различаются по индексу (ID), оставляем только первую.
- Это снижает искусственное увеличение веса отдельных примеров в обучении.

In [18]:
def drop_duplicates_and_conflicts(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    """
    Удаляет полные и конфликтные дубликаты и печатает статистику.
    """
    df = df.copy()

    # 1) Полные дубликаты
    full_dupes = df.duplicated(keep="first")
    n_full_dupes = full_dupes.sum()
    df = df[~full_dupes]

    # 2) Конфликтные дубликаты (одинаковые фичи, разный таргет)
    feature_cols = [c for c in df.columns if c != target_col]
    grp = df.groupby(feature_cols, dropna=False)[target_col].nunique()
    conflict_keys = grp[grp > 1].index
    n_conflict_groups = len(conflict_keys)

    if n_conflict_groups > 0:
        conflict_df = pd.DataFrame(list(conflict_keys), columns=feature_cols)
        df = df.merge(conflict_df.assign(__conflict__=1), how="left", on=feature_cols)
        df = df[df["__conflict__"].isna()].drop(columns="__conflict__")

    # 3) Неявные дубликаты (одинаковые фичи + одинаковый таргет)
    feature_cols = [c for c in df.columns if c != target_col]
    same_dupes = df.duplicated(subset=feature_cols + [target_col], keep="first")
    n_same_dupes = same_dupes.sum()
    df = df[~same_dupes]

    # --- Вывод результатов ---
    print("Результат проверки для текущих данных")
    print(f"Полные дубликаты: {n_full_dupes} записей.")
    print(f"Дубликаты по признакам с разными целевыми значениями: {n_conflict_groups} групп.")
    print(f"Неявные дубликаты (одинаковые признаки + одинаковый целевой): {n_same_dupes} записей.")

    return df

In [19]:
heart_train = drop_duplicates_and_conflicts(heart_train, 'heart_attack_risk_binary')

Результат проверки для текущих данных
Полные дубликаты: 0 записей.
Дубликаты по признакам с разными целевыми значениями: 0 групп.
Неявные дубликаты (одинаковые признаки + одинаковый целевой): 0 записей.


Таким образом, в данных нет явных или скрытых дубликатов, которые требовали бы удаления, однако проверка включена в предобработку для предотвращения подобных проблем при поступлении новых данных.

##### Нормализация столбца gender

In [20]:
heart_train['gender'].value_counts()

gender
Male      5882
Female    2560
1.0        156
0.0         87
Name: count, dtype: int64

In [21]:
heart_test['gender'].value_counts()

gender
Male      650
Female    285
1.0        22
0.0         9
Name: count, dtype: int64

В исходных данных признак gender представлен в смешанном формате: часть записей — строковые значения (Male/Female), часть — числовые (0, 1, 0.0, 1.0). Такой разнородный тип данных может вызвать ошибки или некорректное поведение при автоматической обработке в пайплайне.

Таким образом, принято решение закодировать gender вручную в числовой бинарный формат (1 — мужчина, 0 — женщина) до подачи данных в модель.

In [22]:
def normalize_gender(series: pd.Series) -> pd.Series:
    """
    male/female, '1'/'0', '1.0'/'0.0' -> {1.0, 0.0}, NaN сохраняем
    """
    s = series.astype(str).str.strip().str.lower()
    s = (s.replace({"male":"1","female":"0"})
           .str.replace(".0","", regex=False)
           .replace({"nan": np.nan}))
    return pd.to_numeric(s, errors="coerce")

In [23]:
if "gender" in heart_train.columns:
    heart_train["gender"] = normalize_gender(heart_train["gender"])
if "gender" in heart_test.columns:
    heart_test["gender"] = normalize_gender(heart_test["gender"])

In [24]:
heart_train['gender'].value_counts()

gender
1    6038
0    2647
Name: count, dtype: int64

In [25]:
heart_test['gender'].value_counts()

gender
1    672
0    294
Name: count, dtype: int64

Таким образом, gender был преобразован в единый числовой флаг, что делает обработку данных более надёжной и предсказуемой для всех типов моделей, включая алгоритмы деревьев и линейные классификаторы.

##### Изменение типа данных

Для бинарных, категориальных и порядковых признаков надёжнее переводить значения в int перед кодированием — это исключает неоднозначную интерпретацию и проблемы с обработкой, а так же уберет визуальный шум при построении графиков.

In [26]:
int_features = [
    'diabetes',
    'family_history',
    'smoking',
    'obesity',
    'alcohol_consumption',
    'previous_heart_problems',
    'medication_use',
    'gender',
    
    # Порядковые признаки
    'diet',
    'stress_level',
    'physical_activity_days_per_week'
]

# Приведение к int
for col in int_features:
    heart_train[col] = heart_train[col].dropna().astype(int)
    heart_test[col] = heart_test[col].dropna().astype(int)

In [27]:
heart_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8685 entries, 2664 to 7270
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   age                              8685 non-null   float64
 1   cholesterol                      8685 non-null   float64
 2   heart_rate                       8685 non-null   float64
 3   diabetes                         8442 non-null   float64
 4   family_history                   8442 non-null   float64
 5   smoking                          8442 non-null   float64
 6   obesity                          8442 non-null   float64
 7   alcohol_consumption              8442 non-null   float64
 8   exercise_hours_per_week          8685 non-null   float64
 9   diet                             8685 non-null   int32  
 10  previous_heart_problems          8442 non-null   float64
 11  medication_use                   8442 non-null   float64
 12  stress_level          